Build out triplets for the SBert Finetuning for minimizing triplet loss
Create Separate TF/IDF models for Entities Lemmas and Body Tokens for use on corresponding vectorization

Train the model with the instances to get the learned weights to use for clustering

<b> This notebook relies on the build_repclusters notebook to create the necessary representations from the test set</b>

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import sys
sys.path.insert(1, './drive/MyDrive/EventClustering/src')


In [ ]:
#!pip install transformers
#!pip install bert-tensorflow
!pip install tensorflow

In [8]:
import pandas as pd
import utils
import corpus
import cluster
import model

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

lang='eng'

Pytorch Version: 1.12.0+cu113


ValueError: ignored

In [ ]:
lang = 'eng'
test_corpus = corpus.Corpus()
test_corpus.load_corpora(r"dataset/dataset.dev.json",
                           r"dataset/clustering.dev.json", set([lang]))

In [ ]:
print(lang,"#docs",len(test_corpus.documents))


In [ ]:
df_testcorpus = pd.DataFrame(test_corpus.documents)

In [ ]:
input_data = test_corpus.documents 
utils.random.shuffle(input_data)
#input_data = input_data[:2500]

In [ ]:
df_clusters=df_testcorpus.sort_values(['id','text'], ascending =[1,0])
df_clusters = df_clusters.groupby('cluster').filter(lambda x : len(x)>2)


In [ ]:
print(len(df_clusters))
print ((df_clusters.head()))

In [ ]:
clusters = df_clusters.cluster.unique()
print (len(clusters))

In [ ]:
%%capture
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

In [ ]:
instances = []
#For every cluster in the training set (using as subset for testing)
#Build the cluster representations of the averages of its documents

with tqdm(total=len(clusters)) as pbar:

  for c in (clusters):
    
    #Generate the list of documents for that cluster and create the full cluster representation
    doclist = df_clusters.loc[df_clusters['cluster'] == c]
    assert (len(doclist) > 2 )
    print ('Assembling list of ', len(doclist), ' documents in the cluster #', c)
 
    first_doc = corpus.Document(doclist.iloc[0])
    doclist = doclist.iloc[1:]
    current_cluster=cluster.Cluster(first_doc, c)
    
    doc_reps = []
    for index, row in doclist.iterrows():
        current_doc = corpus.Document(row)
        doc_reps.append(current_doc)
        current_cluster.add_document(current_doc)
        
    
    #Select a random negative cluster number not the current one
    neg_clusternum = utils.random.choice(clusters)  
    while (neg_clusternum == c):
        neg_clusternum = utils.random.choice(clusters)
      
    #Generate the list of documents for that negative cluster and generate the cluster representation
    negdoclist = df_clusters.loc[df_clusters['cluster'] == neg_clusternum]
    assert (len(negdoclist) > 2 )
    print ('Assembling list of ', len(negdoclist), ' negative example documents in the cluster #', neg_clusternum)
 
    first_negdoc = corpus.Document(negdoclist.iloc[0])
    negdoclist = negdoclist.iloc[1:]
    current_negcluster=cluster.Cluster(first_negdoc, neg_clusternum)
    
    for index, row in negdoclist.iterrows():
        current_negdoc = corpus.Document(row)
        current_negcluster.add_document(current_doc)
 
      
    #For each document in the new cluster create a loss triplet using the current cluster and a negative example cluster 
    for anchor in doc_reps:
        
        #Create the similarity vector with current cluster
        print ('\nPostive sample with cluster ',current_cluster.reprs['cluster'],' and anchor doc cluster ', anchor.reprs['cluster'])
        sims = utils.sim_reps_dc(anchor, current_cluster) 
        sim_vec = utils.create_vec(sims)
        
         
        #Create the similarity vector with negative cluster
        print ('Negative sample with cluster ',current_negcluster.reprs['cluster'],' and anchor doc cluster ', anchor.reprs['cluster'])
        difs = utils.sim_reps_dc(anchor, current_negcluster)
        dif_vec = utils.create_vec(difs)

        loss = utils.np.subtract(sim_vec, dif_vec)
        print ('Their difference: ', loss)
        instances.append([loss, 1])
        
    #cluster_reps.append(current_cluster)
    pbar.update(1)


In [ ]:
print (instances[:20])

In [ ]:
#SAVE THE TEST CLUSTER REPRESENTATIONS WITH PICKLE
with open('POC/models/losstraining_allAug2.pkl','wb') as f:
    utils.pickle.dump(instances, f)


In [ ]:
print (training_reps[3].reprs['cluster'])

print (cluster_reps[3].__dict__['oldest_timestamp'])
print (cluster_reps[3].__dict__['sumsq_timestamp'])
print (cluster_reps[3].reprs['cluster'])

arr = cluster_reps[3].reprs['body']

print (arr[utils.np.nonzero(arr)])
print (cluster_reps[3].__dict__['num_docs'])


In [ ]:
#cluster_reps= pd.read_pickle("POC/models/dev_cluster_pool_2KJul30.pkl")

In [ ]:
#training_reps= pd.read_pickle("POC/models/dev_training_reps_2KJul30.pkl")

In [ ]:
print (len(cluster_reps))

In [ ]:
print (len(training_reps))

In [ ]:
training_reps[0]

In [ ]:
cluster_reps[0].clustid

In [ ]:
clusters = {rep.clustid for rep in cluster_reps}
print (len(clusters))

In [ ]:
print (clusters)

Get the triplet loss value by the similarity differences between the anchor and positive cluster and anchor and negative cluster

In [ ]:
import numpy as np
import random
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()
import random

instances = []
with tqdm(total=len(clusters)) as pbar:

  for c in clusters:
    doclist = list(filter(lambda clst: clst.reprs['cluster'] == c, training_reps))
    target_cluster = list(filter(lambda clst: clst.reprs['cluster'] == c, cluster_reps))
    #ensure the cluster representation is unique
    assert (len(target_cluster) == 1)
    for anchor in doclist:
        print ('\nPostive sample with cluster ',target_cluster[0].reprs['cluster'],' and anchor doc cluster ', anchor.reprs['cluster'])
        sims = utils.sim_reps_dc(anchor, target_cluster[0] )
        
        sim_vec = utils.create_vec(sims)
        print (sim_vec)
        neg_cluster = random.choice(cluster_reps)
        
        while (neg_cluster.reprs['cluster'] == c):
            neg_cluster = random.choice(cluster_reps)
            
        print ('Negative sample with cluster ',neg_cluster.reprs['cluster'],' and anchor doc cluster ', anchor.reprs['cluster'])
        difs = utils.sim_reps_dc(anchor, neg_cluster)
        dif_vec = utils.create_vec(difs)
        print (dif_vec)

        loss = np.subtract(sim_vec, dif_vec)
        print ('Their difference: ', loss)
        instances.append([loss, 1])
    pbar.update(1)


In [ ]:
print (instances[2])

In [ ]:
print (len(instances))

In [ ]:
train_df = pd.DataFrame.from_records(instances, columns=['features', 'labels'])

In [ ]:
train_df.head()

In [ ]:
train_len = (len(train_df))

In [ ]:
print (train_len)

In [ ]:
half = train_len*0.5

In [ ]:
train_df.loc[half:train_len-1,'labels'] =0

In [ ]:
train_df.groupby('labels').count()


In [ ]:
train_df.tail()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df.features, train_df.labels, test_size=0.2, random_state=109) 

Train linear classifier using the loss instances on an artificially balanced dataset

In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(list(X_train), list(y_train))
y_pred = clf.predict(list(X_test))

In [ ]:
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

In [ ]:
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_pred))

# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_pred))

In [ ]:
filename = 'POC/models/SVM_similaritymodel_Aug4.svm'
utils.pickle.dump(clf, open(filename, 'wb'))

In [ ]:
print (model.model_reps + model.time_reps)

In [ ]:
print (dict(zip (model.model_reps + model.time_reps, clf.coef_[0])))

Train separate Tf-Idf models for variations of the text corpus. These variations are tokens, lemmas, and entities occurring in the entire corpus. 
These are used to encode into representations vectors for title, body, title body - and their lemma and entity variations